In [1]:
def W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,Model):
    
    import gensim
    #1__WIKI
    if Model == 'Wiki':
        model = gensim.models.Word2Vec.load("W2V_Wiki_Trastd/wiki.zh.sim.seg.trastd.model")
    #2__搜狐新闻数据
    elif Model == 'SogouCS':
        model = gensim.models.Word2Vec.load('W2V_SogouCS/SogouCS_Combine.model')
    #3__全网新闻数据
    elif Model == 'SogouCA':
        model = gensim.models.Word2Vec.load('W2V_SogouCA/SogouCA_Combine.model')
       
    import operator
    cnt = []#　　計算餵進來的總詞量
    Expt = []#　　存W2V無法辨識的詞語
    Remain = []#　　存沒有通過門檻的詞語
    ClassingResult = {}

    for z in Concept:#　　把概念標頭寫進字典
        ClassingResult[z] = {}#　　每個字典的內容都是一個字典

    for q in open(Corpus,'r'):#　　把萃取關鍵字的結果讀出來        
        q = q.strip().decode('utf-8')#　　去空白,解碼
        cnt.append(q)#　　把讀取內容寫進一陣列(計算樣本的資料量用)
        try:
            TmpDic = {}#　　暫時性的字典(比對詞語與各概念的相似度)
            SameConcept = ''#　　用來存放概念,當q直接是某概念時使用
            TmpWord = 'None'#　　用來比對詞語是不是與概念完全吻合,若是,None會被修改掉;反之則維持None(初值)

            for a in Concept:#　　讀取各概念標頭
                TmpList = []#　　暫時性的陣列(存放'詞語'vs'概念內各詞語'的相似度)     
                for b in Concept[a]:#　　讀取個概念內的詞語

                    #####完全ㄧ樣#####
                    if q.encode('utf-8') == b:#　　如果等於
                        #print '---目標詞「',q,'」就在【',a,'】的概念裡'
                        TmpWord = b#　　把詞語指定給TmpWord
                        break#　　跳出迴圈(已經知道跟某詞語完全一樣,就不用再比其他的詞語了)
                    #####

                    if model.similarity(q,b.decode('utf-8')) > SimLevel:#　　如果相似度達門檻
                        TmpList.append(model.similarity(q,b.decode('utf-8')))#　　就把相似度寫進TmpList裡面

                #####完全一樣#####        
                if TmpWord != 'None':#　　不是None(初值)->被修改過->要完全相等才會出現此情況
                    SameConcept = a#　　把現在比對到的概念( EX:輕鬆 )記起來
                    break#　　跳出迴圈(已經知道是屬於哪個概念,就不用再比對其他概念了)
                #####

                if float(len(TmpList))/float(len(Concept[a])) > SimPercent:#　　如果達到相似成數的門檻
                    #print '  -「',q,'」已過相似_百分比_門檻 :',SimPercent
                    TmpAvg = 0.0#　　存放平均相似度
                    for c in TmpList:#　　把每個相似度讀出來
                        TmpAvg += c#　　加起來
                    TmpDic[a] = float(TmpAvg)/float(len(TmpList))#　　把'總數'除以'陣列長度',得到平均相似度,
                                                                    #再寫進字典裡,當q與多個概念都有相似度食
                                                                    #可以比較與哪個概念的相似度比較高

            #####完全一樣##### 
            if TmpWord != 'None': 
                ClassingResult[SameConcept][(q.encode('utf-8'))] = 1#　　把q寫進總字典的概念裡(Key:q Value:1)
                continue
            #####

            if len(TmpDic) > 0:#　　如果暫存字典有任何一概念(表示q'至少'與一個概念相似)
                #print '> >小結 :',q,'成功歸類！',
                for d in TmpDic:#　　遍歷TmpDic
                    if d == sorted(TmpDic.iteritems(),key=operator.itemgetter(1),reverse=True)[0][0]:
                        #如果d = 排序後的第一名 (表示d就是與q最像的概念)
                        if TmpDic[d] > SimLevel:#　　如果"q與概念的相似度"大於相似度門檻
                            #print '類別為 :',d,'\n'
                            ClassingResult[d][q.encode('utf-8')] = TmpDic[d]#　　把q寫進總字典的概念裡(Key:q Value:相似度)
                        else:#　　不太可能發生的情況
                            Remain.append('WHAT!?'+q.encode('utf-8'))                    
            else:#　　q與各概念相似度都沒有達標,無法歸類
                Remain.append(q.encode('utf-8'))

        except KeyError:#　　抓到KeyError( W2V不認識q )的例外處理
            #print '「',q,'」EXCEPT! (KeyError)\n'
            Expt.append(q.encode('utf-8'))#　　把q加進例外的陣列理
            
#####　　把分類結果字典回傳　　#####
    return ClassingResult

##################################################################################################

##################################################################################################

def AttrClassify(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict,\
                     Concept,Corpus,SimLevel,SimPercent,Model):
    
    import re
    from pprint import pprint
    total = 0#　計算總共有多少行資料
    title = 0#　計算有多少的景點
    length = 0#　計算有多少個關鍵字

    ReAttr = '('+Folder+')(.)(.*)'#　計算表頭，正規化用
    for i in open(KeyWordsFile,'r'):#　打開結果的檔案 
        if re.match(ReAttr,i.strip()):#　計算表頭(來源的資料夾)
            title += 1#　表頭(景點)＋１
        else:#　不是表頭。就是關鍵字
            length += 1#　關鍵字＋１
        total += 1#　全部＋１

    if PrintReport == 'Y':
        print "景點總數:",title," 關鍵字總數:",length," 原始檔長度:",total#　把數字都印出來
        if (title+length) == total:#　如果 (景點+關鍵字) = (總行數)
            print '1. 檔案讀取成功! 並無遺漏!\n'#　代表沒有遺漏！成功讀取
        else:
            print '1. 錯誤! 請檢查!\n'

    domain = Folder+'\\'#  景點評論所在的資料夾(同時也是分割內容的標準)

    with open(KeyWordsFile,'r') as fid:#　再讀一次結果的檔案
        content = fid.read()#　把內容讀出來，存給content 
        DictA = {}#　創建一字典
        c = 0#　另一變數c 初值為0

        for i in content.split(domain):#　把內容依據CommentSample(存放景點評論的資料夾名稱)切割
            if c > 0:#　因為split的結果，第0個(也就是for迴圈第一次進來)的值會是空的，所以至少要第二次進來的我才要
                d = 0#　另一變數d 初值為0
                Now = ''

                for o in i.split():#　用空白把每個資料切開
                    if d == 0:#　split的結果，第0個會是檔名(EX: 15.txt)，並不是關鍵字，要特別處理
                        Now = o.strip()#　如果是表頭的話，就把文字指定給外面的變數Now
                        DictA[Now] = []#　把表頭新增進字典裡，並指定內容為一陣列
                    else:#　關鍵字都會進來這裡
                        DictA[Now].append(o.strip())#　把關鍵字加進字典的Value陣列裡
                    d+=1#　做完之後，把d加一。目的僅是因為首次進來(d=0)時，要為字典新增KEY
                        #  以便之後的關鍵字有家可歸，之後繼續加上去並無實值意義
            c+=1#　做完之後，把c加一。目的僅是為了避免第一次進來(c=0)的情況，之後繼續加上去並無實值意義

    ######　　做完以上動作，會得到一字典DictA，KEY:(各景點編號.txt) Value:(各景點的關鍵字)

    DictB = {}#  創建一字典(目的是要把DictA的KEY)轉換成中文
    cnt = 0#  計算關鍵字數量用

    for a in DictA:#  把字典的KEY(EX: 15.txt)讀出來
        count = 0#  另一變數初值為 0
        for u in open(domain+a,'r'):#  (資料夾+字典的KEY) = 檔案的相對位置；把檔讀出來 
            count+=1#  每次進來都先+1，表是讀到第 N 行
            if count == 2:#  當count = 2(讀到第二行, 也就是景點名稱)時，(假設讀到15.txt, 第二行是'陽明山')
                DictB[u.strip()] = DictA[a]#  就把(15.txt)的Value指定給新的字典的'陽明山'這個KEY，達到檔名轉中文的目的
                break#  得到中文景點名稱之後，就沒有必要讀下去，直接中斷此次迴圈
        for b in DictA[a]:#  把每個KEY的關鍵字讀出來
            cnt += 1#  計算關鍵字數量

        #####印出原始字典(DictA)#####
        if PrintDict == 'PrintA':
            print a,'\n',str(DictA[a]).decode('string_escape')
        ###########################

    if PrintReport == 'Y':
        print 'DictA內含詞語總量: ',cnt
        if cnt == length:
            print '2. DictA寫入成功! 並無遺漏!\n'
        else:
            print '2. 錯誤! DictA寫入過程出現遺漏!\n'


    cnt2 = 0#  計算關鍵字數量用
    #####印出最終字典#####
    for g in DictB:
        if PrintDict == 'PrintB':
            print g,'\n',str(DictB[g]).decode('string_escape') 
    #####################
        for r in DictB[g]:
            cnt2 += 1

    if PrintReport == 'Y':
        print '\nDictB內含詞語總量: ',cnt2
        if cnt2 == cnt:
            print '3. DictB寫入(轉換)成功! 並無遺漏!\n'
        else:
            print '3. 錯誤! DictB寫入(轉換)過程出現遺漏!\n'  
            
            
###***************************************************
    DictFinal = DictB
    ClassingResult=W2V_ClassificationMethod2(Concept,Corpus,SimLevel,SimPercent,Model)    
###***************************************************

    if PrintDict == 'PrintConcept':
        for ee in ClassingResult:
            print ee,str(ClassingResult[ee]).decode('string_escape')
    
    
    DictResult={}#　把景點關鍵字的結果轉成'概念'.'文字雲'
    for a in DictFinal:#　把{景點:[關鍵字]}的字典讀出來
        DictResult[a] = {'概　念':{},'文字雲':[]}#　指定KEY:景點 Value:概念{} 文字雲

        for b in DictFinal[a]:#　把每個景點的關鍵字讀出來
            z = 'Homeless'#　先假設每個關鍵字都沒有家

            for c in ClassingResult:#　讀取每個概念,內容是{概念:{詞語:比重}}讀出來
                if b in ClassingResult[c]:#　如果b(現在讀到的這個關鍵字)在某概念裡面
                    z = 'Home'#　有家了

                    for e in ClassingResult[c]:#　把概念的詞都讀一讀
                        if b == e:#　如果'概念內的詞'='現在的關鍵字'-->抓到了!
                            if not c in DictResult[a]['概　念']:#　如果景點還沒有此概念
                                DictResult[a]['概　念'][c]={}#　為景點的此概念新增一個字典
                                DictResult[a]['概　念'][c][b] = ClassingResult[c][e]# 並且把字詞寫進此景點的概念裡,KEY:字詞 Value:字詞與概念的相似度    
                            else:#　如果景點已經有此概念了
                                DictResult[a]['概　念'][c][b] = ClassingResult[c][e]#　把字詞寫進此景點的概念裡,KEY:字詞 Value:字詞與概念的相似度
                            break#　因為以經分好家了,寫完之後就跳出迴圈
                    break#　因為已分好家,所以再次跳出迴圈

            if z == 'Homeless':#　如果字詞並沒有屬於任何概念,依然沒有家
                DictResult[a]['文字雲'].append(b)#　就把字詞寫進該景點的'文字雲'裡面
                
                
    #驗證結果
    if PrintReport == 'Y':
        Class = 0
        Cloud = 0
        for g in DictResult:
            for h in DictResult[g]:
                if h == '文字雲':
                    Cloud += len(DictResult[g][h])
                if h == '概　念':
                    for k in DictResult[g][h]:
                        Class += len(DictResult[g][h][k])
                        
        print 'DictReSult內含詞語總量: ',(Class + Cloud)
        if (Class + Cloud) == cnt2:
            print '4. DictReSult寫入(轉換)成功! 並無遺漏!\n'
        else:
            print '4. 錯誤! DictReSult寫入(轉換)過程出現遺漏!\n'


    if PrintDict == 'PrintC':
        for g in DictResult:
            print '【',g,'】',
            for u in DictResult[g]['概　念']:
                print u,
            print '\n'        
            
    if PrintDict == 'PrintD':
        for g in DictResult:
            print '【',g,'】'
            for h in DictResult[g]:
                print '｢',h,'｣'

                if h == '概　念':
                    for k in DictResult[g][h]:
                        print k,len(DictResult[g][h][k]),str(DictResult[g][h][k]).decode('string_escape')
                elif h == '文字雲':
                    print str(DictResult[g][h]).decode('string_escape')
            print ''

    ###決定是否回傳字典
    
    if ReturnDict == 'ReturnA':
        return DictA
    elif ReturnDict == 'ReturnB':
        return DictB
    elif ReturnDict == 'ReturnC':
        return DictResult
    elif ReturnDict == 'ReturnConcept':
        return ClassingResult

In [2]:
Dic = {
'藝文':['藝術','美術','戲劇','音樂'],\
    
'信仰':['信仰','祭祀','祭拜','膜拜','信眾','信奉'],\
    
'美食':['美食','爽口','美味','好吃'],\
'新奇':['新奇','新穎','獵奇','獨特','驚喜'],\

'輕鬆':['輕鬆','休閒','遊憩','悠閒','休憩','消遣','愜意','放鬆'],\
'遊玩':['遊玩','好玩','遊樂'],\
'快樂':['快樂','愉快','開心','歡樂','幸福','美好'],\
'有趣':['有趣','趣味'],\

'脫俗':['脫俗','幽雅','清幽','優雅','氣質','心曠神怡'],\
'樂活':['環保','綠化','清新'],\

'安靜':['安靜','寧靜'],\
'乾淨':['乾淨','整潔','潔淨'],\

'懷舊':['懷舊','復古','情調'],\
'古老':['古老','悠久'],\

'髒亂':['骯髒','很髒','髒亂','低劣','灰塵','破舊'],\

'購物':['商圈','百貨','商場'],\
'特產':['特產','紀念品'],\

'運動':['運動','健康'],\

'漂亮':['漂亮','可愛','美麗','優美'],\
'文青':['清新','優雅','氣質'],\

'壯觀':['雄偉','壯觀','壯麗']
}

# ---實作---

# [1] 確定有一評論資料夾
資料夾內為若干TXT檔，各檔的檔名是一景點的景點編號(EX: 15.txt)。

內容則各存放「單一景點」的評論

各檔內容為：「第一行」空白；「第二行」是景點的中文名稱；

           「第三行之後」每一行都是一筆評論。編碼UTF-8

# [2] 先確定有關鍵字的文字檔
打開－「Method_V2_NoEng_Write_4PKResult.ipynb」

使用－「FourWayPK()」輸出的結果(記得要寫標頭-->最後一個參數要是'T')

得到－「NE_SciTfidf」開頭的TXT檔

# [3] 實際操作

AttrClassify(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict,\
             Concept,Corpus,SimLevel,SimPercent,Model)
參數：
    
    1.******
    KeyWordsFile: [字串]。[檔名]。存放關鍵字結果的檔案( 要有標頭 )
    ----->EX: 'NE_SciTfidf-30-20160118.txt'

    2.******
    Folder      : [字串]。[資料夾名]。存放評論的資料夾( 格式要照[1]的規定 )
    ----->EX: 'CommentSample'

    3.******
    PrintReport : [字串]。['Y'or'N']。決定是否印出報告，以確認讀取過程中有無遺漏
    
              'Y'：印出報告
              'N'：不印
              
    4.******
    PrintDict   : [字串]。['PrintA'or'PrintB'or'N']。決定是否印出字典來看看

              'PrintA'：印出DictA。
                        KEY: 景點編號.txt　Value: 景點關鍵字
                        
              'PrintB'：印出DictB。(與DictA完全相同,但KEY改成中文)
                        KEY: 景點名稱　    Value: 景點關鍵字
                        
              'PrintC'：印出DictResult(簡單版)
                        KEY: 景點名稱　    Value: 景點貼上的標籤
                        
              'PrintD'：印出DictResult(完整版)
                        KEY: 景點名稱　    Value: 景點被貼上的標籤
                                                 文字雲(景點未被貼標籤的關鍵字)
                                                 
              'PrintConcept'：印出ClassingResult。關鍵字透過W2V分完類的結果
                              KEY: 概念　　Value: 關鍵字
                              
              'N'：不印
              
    5.******
    ReturnDict  : [字串]。['ReturnA'or'ReturnB'or'N']。決定是否回傳字典
    
              'ReturnA'：回傳DictA。
                         KEY: 景點編號.txt　Value: 景點關鍵字
                         
              'ReturnB'：回傳DictB。(與DictA完全相同,但KEY改成中文)
                         KEY: 景點名稱　    Value: 景點關鍵字
                         
              'ReturnC'：回傳DictResult(完整版)
                         KEY: 景點名稱　    Value: 景點被貼上的標籤
                                                  文字雲(景點未被貼標籤的關鍵字)
                                                 
              'ReturnConcept'：回傳ClassingResult。關鍵字透過W2V分完類的結果
                               KEY: 概念　　Value: 關鍵字
                               
              'N'：不回傳
              
              【注意！】此參數是為了可以連接其他方法使用！！！(EX. 寫成JSON檔)
                       在使用時PrintReport . PrintDict都必須設成'N'才可以！！

    6.******
    Concept   :[字典]。概念字典。
               格式為：Key  : 概念的名稱(僅命名,無實質意義)
                      Value: [陣列],內容是可以代表該概念的詞語
               
    7.******
    Corpus    :[字串]。[檔名]。存取關鍵字的TXT檔。一行一個。
               注意！檔案內第一行要空著！編碼要是UTF-8
               原則上同第一個參數 KeyWordsFile 的內容就可以
                
    8.******
    SimLevel  :[浮點數]。[0~1]。相似度門檻。

    9.******
    SimPercent:[浮點數]。[0~1]。相似成數(百分比)門檻。
               必須與概念內詞語相似達到一定程度才能算與此概念相似。

    10.******
    Model     :[字串]。決定要使用哪個W2V的Model
               'Wiki'   :中文WIKI
               'SogouCS':搜狐新闻数据(08年)
               'SogouCA':全网新闻数据(08年)
               
使用範例：
    
    (I)寫好參數
    KeyWordsFile = 'NE_SciTfidf-30-20160118.txt'　<--Sci-TFIDF的結果(有標頭)
    Folder = 'CommentSample'　<--存放評論的資料夾
    PrintReport = 'N'　<--不要印報告
    PrintDict = 'N'　<--不要印字典
    ReturnDict = 'ReturnC'　<--回傳分類完的字典

    Concept   =Dic　<--自訂義的概念字典
    Corpus    ='NE_SciTfidf-30-20160118.txt'　<--Sci-TFIDF的結果(有標頭)
    SimLevel  =0.35　<--相似度門檻
    SimPercent=0.49　<--相似度成數門檻
    Model     ='Wiki'　<--選擇要使用維基的Model
    
    (II)用個變數把結果接住
    ResultDic =
    AttrClassify(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict,\
                 Concept,Corpus,SimLevel,SimPercent,Model)
    
    (III)接下來就可以把這個結果拿來使用/寫成JSON檔/放進MongoDB...etc

In [3]:
import time
a = time.time()

#(I)設定參數
KeyWordsFile = 'NE_SciTfidf-30-20160118.txt'
Folder = 'CommentSample'
PrintReport = 'Y'
PrintDict = 'N'
ReturnDict = 'N'

Concept   =Dic
Corpus    ='NE_SciTfidf-30-20160118.txt'
SimLevel  =0.35
SimPercent=0.49
Model     ='Wiki'

#(II)使用方法
AttrClassify(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict,Concept,Corpus,SimLevel,SimPercent,Model)

b = time.time()
print '耗時',b-a,'秒!'


景點總數: 178  關鍵字總數: 3848  原始檔長度: 4026
1. 檔案讀取成功! 並無遺漏!

DictA內含詞語總量:  3848
2. DictA寫入成功! 並無遺漏!


DictB內含詞語總量:  3848
3. DictB寫入(轉換)成功! 並無遺漏!

DictReSult內含詞語總量:  3848
4. DictReSult寫入(轉換)成功! 並無遺漏!

耗時 10.0160000324 秒!


# 寫成JSON檔

In [18]:
import json

#把結果接住
Result = AttrClassify(KeyWordsFile,Folder,PrintReport,PrintDict,ReturnDict,Concept,Corpus,SimLevel,SimPercent,Model)

json_str = json.dumps(Result)#　丟進一JSON格式的變數

JSONFILE = 'DictConcept.json'#　檔案命名

with open(JSONFILE,'w') as jfid:#　寫進Json檔裡
    jfid.write(json_str)